## Projeto PBD

In [1]:
import os
import sys

import numpy as np 
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
# from ydata_profiling import ProfileReport
import warnings
warnings.filterwarnings("ignore")

Intitializing PySpark ...

23/03/23 10:17:43 WARN Utils: Your hostname, DESKTOP-M9FP7G5 resolves to a loopback address: 127.0.1.1; using 192.168.89.130 instead (on interface eth0)
23/03/23 10:17:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/23 10:17:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Web UI available at http://192.168.89.130:4040
SparkContext available as 'sc' (version = 3.3.1, master = local[*], app id = local-1679566669565)
SparkSession available as 'spark'


In [2]:
import sys

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

from pyspark.ml import Pipeline
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [3]:
from pyspark.sql import SparkSession

# build our own SparkSession
spark = SparkSession\
    .builder\
    .appName("BigData")\
    .config("spark.sql.shuffle.partitions",6)\
    .config("spark.sql.repl.eagereval.enabled",True)\
    .getOrCreate() 

23/03/23 10:18:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
spark

### 1ªInicio
* Read CSV
* Perform EDA

	- Understand the problem
	- Which one makes sense to keep for Ranking

In [5]:
! head -n 2 train.csv
! tail -n 1 train.csv

date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,9,2014-08-27,2014-08-31,2,0,1,8250,1,0,3,2,50,628,1

2014-09-18 08:52:42,2,3,66,462,12565,106.4274,1198182,0,0,0,2014-09-18,2014-09-19,1,0,1,18811,1,1,1,2,50,592,42



In [6]:
filename = 'train.csv'
df_hotels =  spark.read.csv(filename, header=True, inferSchema="true", sep=',')

### Check DATA

In [7]:
df_hotels.printSchema()
df_hotels.show(1, vertical=True)
hotels_count = df_hotels.count()
hotels_count

root
 |-- date_time: timestamp (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- user_location_region: integer (nullable = true)
 |-- user_location_city: integer (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- is_mobile: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- srch_ci: timestamp (nullable = true)
 |-- srch_co: timestamp (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_rm_cnt: integer (nullable = true)
 |-- srch_destination_id: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- is_booking: integer (nullable = true)
 |-- cnt: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 |-- hotel_country: 

37670293

### **Evaluate DATA**


### Drop column User_id

In [ ]:
df_hotels = df_hotels.drop('user_id')

In [ ]:
df_hotels.show(1, vertical = True)

### Create a number off nights

In [ ]:
df_hotels=df_hotels.withColumn("n_of_nights", datediff(F.col("srch_co"),F.col("srch_ci"))-1)
df_hotels.show(3,vertical=True)

### **Id hotel**

In [ ]:
df_hotels = df_hotels.withColumn('Id_hotel', 
                    F.concat(F.col('hotel_cluster'),F.lit('_'), F.col('hotel_market'),F.lit('_'), F.col('hotel_country'),F.lit('_'), F.col('hotel_continent')))
df_hotels.show(5,vertical = True)

### **NULLS**

In [8]:
print('\nNulls in Hotels:')
cols_to_forget = ['date_time','srch_ci','srch_co']
hotels_cols_interest = [x for x in df_hotels.columns if x not in cols_to_forget]
for cl in hotels_cols_interest:
    k = df_hotels.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {hotels_count} records ({k*100/hotels_count:.2f}%)')


Nulls in Hotels:
Column orig_destination_distance with 13525001 nulls or NaN, out of 37670293 records (35.90%)


In [19]:
hotels_cols_interest.remove('orig_destination_distance')
df_hotels = hotels_cols_interest.remove('orig_destination_distance')

### **Summary to figure out outliers**

In [9]:
df_hotels.describe(hotels_cols_interest).show(vertical = True)

-RECORD 0----------------------------------------
 summary                   | count               
 site_name                 | 37670293            
 posa_continent            | 37670293            
 user_location_country     | 37670293            
 user_location_region      | 37670293            
 user_location_city        | 37670293            
 orig_destination_distance | 24145292            
 user_id                   | 37670293            
 is_mobile                 | 37670293            
 is_package                | 37670293            
 channel                   | 37670293            
 srch_adults_cnt           | 37670293            
 srch_children_cnt         | 37670293            
 srch_rm_cnt               | 37670293            
 srch_destination_id       | 37670293            
 srch_destination_type_id  | 37670293            
 is_booking                | 37670293            
 cnt                       | 37670293            
 hotel_continent           | 37670293            


### **UNIQUENESS**

In [10]:
print('\nUniqueness in Hotels:')
for cl in hotels_cols_interest:
    k = df_hotels.select(cl).distinct().count()
    if k > 0:
        print(f'Column {cl} with {k} distinct values, out of {hotels_count} records ({k*100/hotels_count:.2f}%)')


Uniqueness in Hotels:
Column site_name with 45 distinct values, out of 37670293 records (0.00%)
Column posa_continent with 5 distinct values, out of 37670293 records (0.00%)
Column user_location_country with 237 distinct values, out of 37670293 records (0.00%)
Column user_location_region with 1008 distinct values, out of 37670293 records (0.00%)
Column user_location_city with 50447 distinct values, out of 37670293 records (0.13%)
Column orig_destination_distance with 8495290 distinct values, out of 37670293 records (22.55%)
Column user_id with 1198786 distinct values, out of 37670293 records (3.18%)
Column is_mobile with 2 distinct values, out of 37670293 records (0.00%)
Column is_package with 2 distinct values, out of 37670293 records (0.00%)
Column channel with 11 distinct values, out of 37670293 records (0.00%)
Column srch_adults_cnt with 10 distinct values, out of 37670293 records (0.00%)
Column srch_children_cnt with 10 distinct values, out of 37670293 records (0.00%)
Column srch

### **Saving clean data**


In [13]:
seed = 5
with_replacement = False
fraction = 0.01          
small_df_hotels = df_hotels.sample(withReplacement=with_replacement, 
                                               fraction=fraction, seed=seed)
small_df_hotels.count()

376623

In [14]:
small_df_hotels.write.mode("overwrite").parquet("small-hotels")

### **Final Overview**

In [15]:
df_clean = small_df_hotels

In [16]:
hotels_count = df_clean.count()
hotels_count

376623

In [17]:
cols_to_forget = ['date_time','srch_ci','srch_co']
hotels_cols_interest = [x for x in df_clean.columns if x not in cols_to_forget]
hotels_cols_interest

['site_name', 'posa_continent', 'user_location_country', 'user_location_region', 'user_location_city', 'orig_destination_distance', 'user_id', 'is_mobile', 'is_package', 'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id', 'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market', 'hotel_cluster']

### **Descriptive statistics about the data to be used in the model**

In [19]:
print('\nNulls in Transactions:')
for cl in hotels_cols_interest:
    k = df_clean.select(cl).filter(F.col(cl).isNull() | F.isnan(cl)).count()
    if k > 0:
        print(f'Column {cl} with {k} nulls or NaN, out of {hotels_count} records ({k*100/hotels_count:.2f}%)')


Nulls in Transactions:
Column orig_destination_distance with 135295 nulls or NaN, out of 376623 records (35.92%)


In [21]:
df_clean.printSchema()

root
 |-- date_time: timestamp (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- user_location_region: integer (nullable = true)
 |-- user_location_city: integer (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- is_mobile: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- srch_ci: timestamp (nullable = true)
 |-- srch_co: timestamp (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_rm_cnt: integer (nullable = true)
 |-- srch_destination_id: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- is_booking: integer (nullable = true)
 |-- cnt: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 |-- hotel_country: 

In [24]:
# few distincts
cls_1 = ['site_name', 'user_id', 'is_mobile', 'srch_adults_cnt', 'hotel_cluster', 'is_booking']

# numeric types but no nulls
cls_2 = ['site_name', 'user_id', 'is_mobile', 'srch_adults_cnt', 'hotel_cluster', 'is_booking', 'hotel_country', 'srch_destination_id', 'is_package']

In [25]:
print('\nUniqueness in Transactions:')
for cl in cls_1:
    df_clean.select(cl).distinct().sort(cl).show(truncate=False)


Uniqueness in Transactions:
+---------+
|site_name|
+---------+
|2        |
|5        |
|6        |
|7        |
|8        |
|9        |
|10       |
|11       |
|13       |
|14       |
|15       |
|16       |
|17       |
|18       |
|19       |
|20       |
|21       |
|22       |
|23       |
|24       |
+---------+
only showing top 20 rows

+-------+
|user_id|
+-------+
|4      |
|8      |
|11     |
|13     |
|15     |
|31     |
|36     |
|40     |
|47     |
|52     |
|62     |
|74     |
|81     |
|86     |
|89     |
|103    |
|105    |
|114    |
|116    |
|119    |
+-------+
only showing top 20 rows

+---------+
|is_mobile|
+---------+
|0        |
|1        |
+---------+

+---------------+
|srch_adults_cnt|
+---------------+
|0              |
|1              |
|2              |
|3              |
|4              |
|5              |
|6              |
|7              |
|8              |
|9              |
+---------------+

+-------------+
|hotel_cluster|
+-------------+
|0            |
|

### **Correlations**

In [26]:
def plotHistogram(df, xcol, huecol=None):
    sns.histplot(data=df, x=xcol, hue=huecol, multiple="stack")

In [27]:
def plot(df, xcol, ycol):
    sns.lineplot(data=df, x=xcol, y=ycol)

In [28]:
def plotBar(df, xcol, ycol, huecol=None):
    sns.barplot(data=df, x=xcol, y=ycol, hue=huecol)

In [29]:
def plotScatter(df, xcol, ycol, huecol=None):
    sns.scatterplot(data=df, x=xcol, y=ycol, hue=huecol)

In [30]:
def plotScatterMatrix(df, huecol=None):
    sns.pairplot(data=df, hue=huecol)

In [31]:
def plotCorrelationMatrix_1(df, annot=False):
    # compute the correlation matrix
    corr = df.corr()
    
    # generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))

    # generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    #cmap='coolwarm'

    # draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, annot=annot,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [32]:
def plotCorrelationMatrix_2(df):
    # compute a correlation matrix and convert to long-form
    corr_mat = df.corr().stack().reset_index(name="correlation")
    # draw each cell as a scatter point with varying size and color
    g = sns.relplot(
        data=corr_mat,
        x="level_0", y="level_1", hue="correlation", size="correlation",
        palette="vlag", hue_norm=(-1, 1), edgecolor=".7",
        height=10, sizes=(50, 250), size_norm=(-.2, .8),
    )

    # tweak the figure to finalize
    g.set(xlabel="", ylabel="", aspect="equal")
    g.despine(left=True, bottom=True)
    g.ax.margins(.02)
    for label in g.ax.get_xticklabels():
        label.set_rotation(90)
    for artist in g.legend.legendHandles:
        artist.set_edgecolor(".7")

In [33]:
cols_corr = cls_2

vector_col = "corr_features"
assembler = VectorAssembler(inputCols=cols_corr, outputCol=vector_col)
df_vector = assembler.transform(df_clean).select(vector_col)

# get correlation matrix - it can be Pearson’s (default) or Spearman’s correlation

# corr = Correlation.corr(df_vector, vector_col).head()
# print("Pearson correlation matrix:\n" + str(corr[0]))

# corr = Correlation.corr(df_vector, vector_col, "spearman").head()
# print("Spearman correlation matrix:\n" + str(corr[0]))

corr_matrix = Correlation.corr(df_vector, vector_col).collect()[0][0].toArray().tolist()

23/03/23 11:57:27 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/03/23 11:57:27 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [34]:
df_plot = pd.DataFrame(data = corr_matrix, index=cols_corr, columns=cols_corr)
plotCorrelationMatrix_1(df_plot, annot=True)
plt.title('Correlations among numerical features')
plt.show()

In [36]:
df_plot = ( df_clean
                .groupby('hotel_cluster')
                .count()
                .sort('hotel_cluster', ascending=True)
                .toPandas()
          )
plotBar(df_plot, 'hotel_cluster', 'count')
plt.title('Number of transactions by year')
plt.show()
df_plot.head()

,hotel_cluster,count
0,0,3748
1,1,4440
2,2,4406
3,3,2231
4,4,3504


In [ ]:

df_plot = ( df_clean
            .groupby(['Year', 'Month'])
            .count()
            .withColumn('Year - Month', F.concat('Year', 'Month'))
            #.sort('Year - Month', ascending=True)
            .toPandas()
          )
df_plot.head()

### **Train Model**

In [15]:
# Importando as bibliotecas necessárias
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.recommendation import ALS

# Criando um vetor com os atributos que serão usados para fazer as recomendações
vectorAssembler = VectorAssembler(inputCols=["srch_children_cnt", "is_mobile", "is_booking", "is_package", "user_location_country"], outputCol="features")
df_hotels = vectorAssembler.transform(df_hotels)

# Separando os dados em conjuntos de treinamento e teste
(training, test) = df_hotels.randomSplit([0.8, 0.2])

# Configurando o modelo ALS (Alternating Least Squares) para fazer as recomendações
als = ALS(userCol="user_id", itemCol="hotel_cluster", ratingCol="is_booking", coldStartStrategy="drop")
model = als.fit(training)

# Fazendo as recomendações para cada usuário com base em seus históricos de reservas e outros atributos
recommendations = model.recommendForAllUsers(10)

# Mostrando as recomendações para um usuário específico
recommendations.filter(recommendations.user_id == 1234).show()

23/03/22 19:53:02 WARN BlockManager: Block rdd_403_2 could not be removed as it was not found on disk or in memory
23/03/22 19:53:02 WARN BlockManager: Block rdd_404_2 could not be removed as it was not found on disk or in memory
23/03/22 19:53:02 WARN BlockManager: Block rdd_403_1 could not be removed as it was not found on disk or in memory
23/03/22 19:53:02 WARN BlockManager: Block rdd_404_1 could not be removed as it was not found on disk or in memory
23/03/22 19:53:02 ERROR Executor: Exception in task 2.0 in stage 201.0 (TID 1540)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3865)
	at scala.collection.mutable.ArrayBuilder$ofFloat.mkArray(ArrayBuilder.scala:471)
	at scala.collection.mutable.ArrayBuilder$ofFloat.resize(ArrayBuilder.scala:475)
	at scala.collection.mutable.ArrayBuilder$ofFloat.ensureSize(ArrayBuilder.scala:487)
	at scala.collection.mutable.ArrayBuilder$ofFloat.$plus$eq(ArrayBuilder.scala:492)
	at scala.collection.mutabl

Py4JJavaError: ('An error occurred while calling o381.fit.\n', JavaObject id=o433)